### Description
This document shows how to normalize the original dataset. It is just used to do the initial data analysis, so only a few of the features are chosen. It chooses itapudid, max1stdetectwssc, max1stdetectwssd, max1stdetectwsse, max1stdetectwssf and eventtime as features (No max1stdetectwssa and max1stdetectwssb because of sensors in that positions). It removes the poweroffevents with all NAN values which have the same itapudid. In the final outcome dataset, there are 3 columns named itapudid, speed, sensor.

In [48]:
import pandas as pd

In [49]:
df = pd.read_pickle('../data/wss')

In [50]:
results = pd.DataFrame(df)
results.head()

,accountid,avgload,avgspeed,bssid,bssid1,bssid2,bssid3,created_at,eventtime,drivenum,...,p1presspoweroff,padwearstat,schemavers,servprvder,softgpsloc,totaldist,tripdist,warnlampstat,itappin,itapudid
0,A3J43,NaN,1,BC765EEC523A,,,,2017-04-24 11:44:20.365,2016-12-12 13:33:12,5,...,6200.0,NaN,3,1,,0.0,0.0,1,1888062301,163540005001DC915C983
1,A3J43,NaN,1,BC765EEC523A,,,,2017-04-24 11:44:20.365,2016-12-12 13:38:10,9,...,6200.0,NaN,3,1,,0.0,0.0,1,1888062301,163540005001DC915C983
2,A3J43,NaN,255,BC765EEC523A,,,,2017-04-24 11:44:20.365,2016-12-12 16:31:04,14,...,7500.0,NaN,3,1,,0.0,0.0,0,1888062301,163540005001DC915C983
3,A3J43,NaN,255,BC765EEC523A,,,,2017-04-24 11:44:20.365,2017-01-29 08:56:22,18,...,4500.0,NaN,3,1,,0.0,0.0,0,1888062301,163540005001DC915C983
4,A3J43,NaN,1,BC765EEC523A,,,,2017-04-24 11:44:42.082,2016-12-12 14:04:23,4,...,5900.0,NaN,3,1,,0.0,0.0,1,4099311238,163540006001DC915B4EF


In [51]:
#select the features and fill in the na values with 0
pd.set_option('display.max_rows', 50)
results = results[['itapudid', 'max1stdetectwssc', 'max1stdetectwssd', 'max1stdetectwsse', 'max1stdetectwssf', 'eventtime']]
results.to_pickle('../data/wss_n1')

In [52]:
df_n_nd = results[results.itapudid != '170540055001DC915C90E']
df_n3 = results[results.itapudid == '170540055001DC915C90E'] #The trailer which has 4 wss

In [53]:
df_nc = df_n_nd[['itapudid', 'max1stdetectwssc', 'eventtime']]
df_nc = df_nc.dropna(axis=0, how='any')
df_nc['sensor'] = 'C'
df_nc.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

df_nd = df_n_nd[['itapudid', 'max1stdetectwssd', 'eventtime']]
df_nd = df_nd.dropna(axis=0, how='any')
df_nd['sensor'] = 'D'
df_nd.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

df_ne = df_n_nd[['itapudid', 'max1stdetectwsse', 'eventtime']]
df_ne = df_ne.dropna(axis=0, how='any')
df_ne['sensor'] = 'E'
df_ne.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

df_nf = df_n_nd[['itapudid', 'max1stdetectwssf', 'eventtime']]
df_nf = df_nf.dropna(axis=0, how='any')
df_nf['sensor'] = 'F'
df_nf.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

In [54]:
frame1 = [df_nc, df_nd]
frame2 = [df_ne, df_nf]
re1 = pd.concat(frame1)
re2 = pd.concat(frame2)

In [55]:
test1 = re1.sort_values(['eventtime','itapudid'])
test2 = re2.sort_values(['eventtime','itapudid'])

In [56]:
test1.to_pickle('../data/wss_n_1')  # pickle dataframe
test2.to_pickle('../data/wss_n_2')  # pickle dataframe

In [57]:
df_cd = pd.read_pickle('../data/wss_n_1')
df_ef = pd.read_pickle('../data/wss_n_2')

In [58]:
import pandasql as pdsql

pysql = lambda q: pdsql.sqldf(q, globals())
str1 = """select 
             itapudid, sum(speed)
        from 
            df_cd
        group by 
            itapudid"""
str2 = """select 
             itapudid, sum(speed)
        from 
            df_ef
        group by 
            itapudid"""

df_cd_1 = pysql(str1)
df_ef_1 = pysql(str2)

In [59]:
df_cd_2 = df_cd_1[df_cd_1['sum(speed)'] == 0].itapudid
df_ef_2 = df_ef_1[df_ef_1['sum(speed)'] == 0].itapudid

In [60]:
#The itapudids where all speeds equal to 0
df_cd_list = df_cd_2.values.tolist()
df_ef_list = df_ef_2.values.tolist()

In [61]:
for i in df_cd_list:
    df_cd = df_cd[df_cd.itapudid != i]

for j in df_ef_list:
    df_ef = df_ef[df_ef.itapudid != j]

In [62]:
df = pd.read_pickle('../data/wss_n1')
df = df[df.itapudid =='170540055001DC915C90E']

In [63]:
df_nc = df[['itapudid', 'max1stdetectwssc', 'eventtime']]
df_nc = df_nc.dropna(axis=0, how='any')
df_nc['sensor'] = 'C'
df_nc.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

df_nd = df[['itapudid', 'max1stdetectwssd', 'eventtime']]
df_nd = df_nd.dropna(axis=0, how='any')
df_nd['sensor'] = 'D'
df_nd.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

df_ne = df[['itapudid', 'max1stdetectwsse', 'eventtime']]
df_ne = df_ne.dropna(axis=0, how='any')
df_ne['sensor'] = 'E'
df_ne.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

df_nf = df[['itapudid', 'max1stdetectwssf', 'eventtime']]
df_nf = df_nf.dropna(axis=0, how='any')
df_nf['sensor'] = 'F'
df_nf.columns = ['itapudid', 'speed', 'eventtime', 'sensor']

In [64]:
frame = [df_nc, df_nd, df_ne, df_nf]
re = pd.concat(frame)
test = re.sort_values(['eventtime','itapudid'])

In [65]:
frame1 = [df_cd, df_ef, test]
re1 = pd.concat(frame1)
datafinal = re1.sort_values(['eventtime','itapudid'])

In [66]:
#The poweroffevent with this eventtime all have 0 speed, remove them
datafinal = datafinal[datafinal['eventtime'] != '1970-01-01 00:00:00'] 

In [67]:
datafinal.to_pickle('../data/wss_n_c1')

In [68]:
datafinal

,itapudid,speed,eventtime,sensor
8568,163540024001DC915C992,0.0,2016-09-07 07:13:37,C
8568,163540024001DC915C992,0.0,2016-09-07 07:13:37,D
4065,163540024001DC915C992,10.0,2016-09-07 07:13:38,C
5246,163540024001DC915C992,0.0,2016-09-07 07:13:38,C
5813,163540024001DC915C992,10.0,2016-09-07 07:13:38,C
5814,163540024001DC915C992,8.0,2016-09-07 07:13:38,C
5820,163540024001DC915C992,10.0,2016-09-07 07:13:38,C
5821,163540024001DC915C992,0.0,2016-09-07 07:13:38,C
7579,163540024001DC915C992,0.0,2016-09-07 07:13:38,C
10701,163540024001DC915C992,9.0,2016-09-07 07:13:38,C
